In [88]:
with open('key.txt') as f:
    key = f.read()

In [91]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import gspread

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'BRL'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': key, #'1111f122-05e2-408c-83d2-c965de3b70ff' >> Esse não vai funcionar
}

session = Session()
session.headers.update(headers)

# response = session.get(url, params=parameters)
# data = json.loads(response.text)

In [92]:
df = pd.DataFrame(data['data'])

In [93]:
info = df.quote.apply(pd.Series)['BRL'].apply(pd.Series)

In [94]:
to_sheet = df[['name', 'symbol', 'slug']]
to_sheet.loc[:, info.columns] = info

In [102]:
to_sheet[to_sheet.columns]

,name,symbol,slug,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
0,Bitcoin,BTC,bitcoin,1.967408e+05,9.514461e+10,-11.3225,0.085279,-2.334279,-11.927389,-0.575479,-21.525288,-34.322234,3.730926e+12,41.9702,4.131556e+12,2022-02-20T14:29:14.000Z
1,Ethereum,ETH,ethereum,1.350947e+04,5.656563e+10,-28.5125,-0.005252,-2.073217,-12.523441,-6.154937,-34.079792,-38.110953,1.616529e+12,18.2375,1.616529e+12,2022-02-20T14:29:14.000Z
2,Tether,USDT,tether,5.141163e+00,2.348041e+11,-8.5690,-0.002069,1.835621,-2.126485,0.029452,0.053839,-0.044358,4.062522e+11,4.5676,4.167668e+11,2022-02-20T14:29:14.000Z
3,BNB,BNB,bnb,1.940808e+03,7.339484e+09,1.4996,-0.500529,-3.281123,-8.929681,-9.535062,-29.217189,-34.695317,3.204600e+11,3.6143,3.204600e+11,2022-02-20T14:29:14.000Z
4,USD Coin,USDC,usd-coin,5.136404e+00,1.273608e+10,-18.2482,-0.042156,1.836029,-2.156279,-0.005105,-0.075804,-0.107004,2.702379e+11,3.0479,2.702379e+11,2022-02-20T14:29:14.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Rematic,RMTX,rematic,2.449628e-08,1.732944e+05,43.8965,-1.402994,-8.133513,-15.125865,-50.597569,-12.379390,-15.193959,0.000000e+00,0.0000,2.283054e+07,2022-02-20T14:29:14.000Z
4996,Sovi Finance,SOVI,sovi-finance,1.052084e+01,1.727903e+05,48.4514,-0.066732,1.617921,-5.940607,1.497178,8.574147,-6.482358,0.000000e+00,0.0000,2.630209e+06,2022-02-20T14:29:14.000Z
4997,Dark Matter DeFi,DMD,dark-matter-defi,3.569645e-01,1.727812e+05,174.1336,-0.431348,-6.948097,-23.577702,2.441621,169.441351,57.478792,0.000000e+00,0.0000,3.034198e+07,2022-02-20T14:29:14.000Z
4998,Dhabi Coin,DBC,dhabi-coin,2.333747e-03,1.724204e+05,13.9288,-0.948849,-21.464310,-41.546997,-97.337123,-98.598058,-98.598058,0.000000e+00,0.0000,7.001242e+06,2022-02-20T14:29:14.000Z


In [103]:
gc = gspread.service_account(filename = '../../key.json')
code = '1F9NY5hQQQozC5MGuseiDxpj218902He_Gna-doOrobs'
sh = gc.open_by_key(code)
ws = sh.worksheet('CoinMarketCap')
clear = ws.clear()
to_sheet = to_sheet.fillna('')
update = ws.update([to_sheet.columns.values.tolist()] + to_sheet.values.tolist())